## Static Train/Val/Test division

In [40]:
# Check the Division Configuration Parameters
import numpy as np
import Functions.MyDataset as MyDataset
import Spartan.support_modules as MySupport


In [6]:
MyDataset.get_pat_splits(static=True)

[array([ 2,  4, 18, 17, 12, 10,  6,  0, 11, 16,  9, 14,  5]),
 array([ 3, 13]),
 array([ 8,  7,  1, 15])]

In [17]:
# file name format: {name}_{size}_VolPts_{id}.mat (AH_17617648_VolPts.mat)
pat_names = ['AH', 'AZ', 'DE', 'DM', 'DM2', 'DGL', 'FA', 'GE', 'GM', 'GP', 'HB', 'HH',
             'JH', 'JM', 'LG', 'LP', 'MJ', 'NV', 'PH', 'SM']

# pat_splits = MyDataset.get_pat_splits(static=True)
pat_splits = []

if not pat_splits:
    np.random.shuffle(pat_names)
    name_splits = np.split(pat_names, [int(.7 * len(pat_names)), int(.8 * len(pat_names))])
else:
    name_splits = [[pat_names[i] for i in pat_splits[0]],
                   [pat_names[i] for i in pat_splits[1]], [pat_names[i] for i in pat_splits[2]]]

print(name_splits)

[array(['SM', 'GP', 'PH', 'DE', 'JH', 'DGL', 'FA', 'LP', 'JM', 'AZ', 'MJ',
       'HB', 'LG', 'GM'], dtype='<U3'), array(['DM2', 'NV'], dtype='<U3'), array(['DM', 'AH', 'HH', 'GE'], dtype='<U3')]


In [56]:
import importlib

importlib.reload(MyDataset)

#pat_splits = MyDataset.get_pat_splits(static=True)
pat_splits = []

if not pat_splits:
    # attention: this division is based on augmentation, no on patients (not sure if it will cause issues)
    idx_list = np.arange(0, 2000, 2)
    np.random.shuffle(idx_list)
    idx_splits = np.split(idx_list, [int(.7 * len(idx_list)), int(.8 * len(idx_list))])
else:
    idx_splits = [[list(range(i*100, i*100+100, 2)) for i in j] for j in pat_splits]
    for i in range(0, 3):
        idx_splits[i] = [num for sublist in idx_splits[i] for num in sublist]
        np.random.shuffle(idx_splits[i])

print(idx_splits)

[array([ 324, 1806, 1914, 1976, 1272,  202,  892,  952, 1656,  764,  698,
        526, 1542,  166, 1278,  326,  928,  284, 1628, 1848,  598, 1886,
        870, 1206, 1518,  810, 1230, 1002,  462,  540, 1156,  878, 1844,
        418, 1768, 1438, 1342, 1740, 1012, 1510, 1450, 1458,  270,  614,
       1540, 1874,  434, 1662, 1534, 1338,  688, 1968, 1574, 1816, 1894,
        420,  802,  990,  872, 1536,  214, 1494,  590,   56,  376,  122,
        904,  308, 1286,  554, 1200, 1064, 1304, 1102, 1794,  814,  516,
        848,  224, 1822, 1560, 1736, 1672, 1368, 1480, 1378,  152,  156,
        970,  404,  250,  230, 1468,  782,  260,  656,  886,  790, 1444,
       1396, 1908,  672,  148,  832, 1382,   88, 1948, 1218, 1670, 1366,
        784,  112,  286, 1606, 1372,  744, 1274, 1234, 1726,  774,  272,
       1068, 1264,  780,  682,  204,  184,  908,  792, 1530,  546,  722,
       1944,  208, 1604, 1696,  754, 1898,  278, 1648, 1592,  174,  522,
       1620, 1352, 1040,  132,  882,  646, 1288, 1